In [1]:
import requests
import json
import pandas as pd

In [20]:
BASE_URL = 'https://api-comexstat.mdic.gov.br/general'

HEADERS = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

POSSIBLE_METRICS = ["FOB", "KG", "Statistic", "Freight", "Insurance", "CIF"]

# NCM stands for "Nomenclatura Comum Mercosul"
# https://portalunico.siscomex.gov.br/classif/#/nesh/consulta?id=114967&dataPesquisa=2025-01-10T19:55:04.000Z&tipoNota=3&tab=11736538995258

NCM_IDS_PREFIX_DICT = {
    'PESTICIDES': '38081',
    'FUNGICIDES': '38082',
    'HERBICIDES': '38083',
    'DESINFETANTES': '38084',
    "UNCLEAR": "38085",  # TODO descobrir oq é o 85
    'OTHERS': '38089',
    "DDT": '29039220',  # NOT A PREFIX ..... but will work 
    "DDT": '29036220',  # NOT A PREFIX .....
}

COLUMN_RENAME_MAP = {
    "coAno": 'ano',
    "coMes": 'mes',
    "noPaispt": "pais",
    "noUf": "estado",
    "noVia": "via_de_transporte",
    "noUrf": "unidade_receita_federal",
    "coNcm": "id_ncm",
    "noNcmpt": "descritor_ncm",
    "noUnid": "unidade_medida",
    "vlFob": "valor_fob_usd",
    "vlFrete": "valor_frete_usd",
    "vlSeguro": "valor_seguro_usd",
    "vlCif": "valor_cif_usd",
    "kgLiquido": "peso_liq_kg",
    "qtEstat": "qtd_estatistica",
}


DEFAULT_FILTER_PARAMS = {
    "typeForm": 2,
    "typeOrder": 1,
    "filterList": [
        {
            "id": "noNcmpt",
            "text": "NCM - Nomenclatura Comum do Mercosul",
            "route": "/pt/product/ncm",
            "type": "2",
            "group": "sh",
            "groupText": "Sistema Harmonizado (SH)",
            "hint": "fieldsForm.general.noNcm.description",
            "placeholder": "NCM",
        }
    ],
    "detailDatabase": [
        {
            "id": "noPaispt",
            "text": "País",
            "group": "gerais",
            "groupText": "Gerais",
        },
        {
            "id": "noUf",
            "text": "UF do Produto",
            "group": "gerais",
            "groupText": "Gerais",
        },
        {
            "id": "noVia",
            "text": "Via",
            "group": "gerais",
            "groupText": "Gerais",
        },
        {
            "id": "noUrf",
            "text": "URF",
            "group": "gerais",
            "groupText": "Gerais",
        },
        {
            "id": "noNcmpt",
            "text": "NCM - Nomenclatura Comum do Mercosul",
            "parentId": "coNcm",
            "parent": "Código NCM",
            "group": "sh",
            "groupText": "Sistema Harmonizado (SH)",
        },
    ],
    "formQueue": "general",
    "langDefault": "pt",
    "monthDetail": 'true',
    "monthStart": "01",
    "monthEnd": "12",
    "monthStartName": "Janeiro",
    "monthEndName": "Dezembro",
}

FOREST_COVERAGE_DATA_URL = "https://dados.florestal.gov.br/pt_BR/api/3/action/datastore_search?resource_id=67d29e7e-0b99-41c5-9586-f0f045bc598c"

In [29]:
def create_forest_coverage_data_df() -> pd.DataFrame:
    r = requests.get(FOREST_COVERAGE_DATA_URL)
    return pd.DataFrame(r.json()['result']['records'])

In [22]:
def get_comexstat_filter_possible_values(
    filter_name: str,
    base_url = BASE_URL,
) -> list:
  endpoint = "filters"

  url = f'{base_url}/{endpoint}/{filter_name}'
  r = requests.get(url, verify=False) ## DO NOT DO THIS

  return r.json()["data"][0]

In [23]:
def create_id_to_classification_map(response_data: list, prefix_dict: dict = NCM_IDS_PREFIX_DICT):
    """
    Create a mapping dictionary from ID to classification.

    Args:
        response_data (list): List of dictionaries containing 'id'.
        prefix_dict (dict): Dictionary of prefixes to match.

    Returns:
        dict: A dictionary mapping IDs to their classification.
    """
    id_to_classification = {}

    for item in response_data:
        item_id = item["id"]  # ID is a string

        for classification, prefix in prefix_dict.items():
            if item_id.startswith(prefix):
                id_to_classification[item["id"]] = classification
                break

    return id_to_classification

In [24]:
def build_query_filter_params(
        ncm_produt_ids: list,
        metrics_columns: list,
        start_year: int,
        end_year: int, 
        default_params: dict,
):
    default_params["filterArray"] = [{"item": ncm_produt_ids, "idInput": "noNcmpt"}]  # do not hardcode  >:(
    default_params["yearStart"] = start_year
    default_params["yearEnd"] = end_year
    for metric in metrics_columns:
        default_params[f'metric{metric}'] = 'true'

    return default_params

In [25]:
def query_defensivos_agricolas_from_comexstat(
      ncm_produt_ids: list,
      metrics_columns: list,
      start_year: int,
      end_year: int, 
      base_url=BASE_URL,
      headers=HEADERS,
      default_params=DEFAULT_FILTER_PARAMS,
):
     assert start_year >= 1997, """Invalid start year. This database starts in 1997"""  # i could check many more things
     
     filter_params = build_query_filter_params(
          ncm_produt_ids=ncm_produt_ids,
          metrics_columns=metrics_columns,
          start_year=start_year,
          end_year=end_year,
          default_params=default_params,
     )

     params = {
           "filter": json.dumps(filter_params)
      }
     try:
      response = requests.get(base_url, headers=headers, params=params, verify=False) ## investigate SSL for this api. No documentation explaining
      response.raise_for_status()  # Raise HTTPError for bad responses
      return response.json()
     
     except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

In [26]:
def create_denfensivos_agricolas_df() -> pd.DataFrame:
    
    possible_ncm_ids = get_comexstat_filter_possible_values(
        filter_name="ncm"
    )
    id_to_classification_map = create_id_to_classification_map(
        response_data=possible_ncm_ids
    )
    interest_ncm_ids_list = list(id_to_classification_map.keys())

    resp = query_defensivos_agricolas_from_comexstat(
        ncm_produt_ids=interest_ncm_ids_list,
        metrics_columns=POSSIBLE_METRICS,
        start_year=1997,
        end_year=2024,
    )

    return pd.DataFrame.from_dict(resp['data']['list']).rename(columns=COLUMN_RENAME_MAP)

In [27]:
df_import = create_denfensivos_agricolas_df()

/home/hcordeiro/personal/venv/dviz/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-comexstat.mdic.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/hcordeiro/personal/venv/dviz/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api-comexstat.mdic.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [30]:
df_forest = create_forest_coverage_data_df()

In [33]:
df_import.columns

Index(['ano', 'mes', 'pais', 'estado', 'via_de_transporte',
       'unidade_receita_federal', 'id_ncm', 'descritor_ncm', 'unidade_medida',
       'valor_fob_usd', 'valor_frete_usd', 'valor_seguro_usd', 'valor_cif_usd',
       'peso_liq_kg', 'qtd_estatistica'],
      dtype='object')

In [32]:
df_forest.head()

,_id,Tipologia_sigla,Tipologia,Categoria,cod_tipo,cod_class,Bioma,Pretérita,1990,2000,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,1,A,Floresta Ombrófila Aberta,Floresta,2,1,Amazônia,"8.966.632,7","6.045.929,0","2.079.538,8",...,"868.820,9","861.966,2","854.919,7","845.724,5","836.916,9","826.789,0","819.925,9","810.886,2","802.390,7","793.895,2"
1,2,A,Floresta Ombrófila Aberta,Floresta,2,1,Caatinga,"11.393,2","7.092,8","6.436,1",...,"4.652,3","4.466,5","4.446,4","4.418,9","4.377,3","4.348,4","4.210,0","4.144,6","4.060,9","3.977,3"
2,3,A,Floresta Ombrófila Aberta,Floresta,2,1,Cerrado,"556,8","384,3","340,1",...,"273,6","273,4","271,7","271,5","268,9","267,1","266,9","265,6","264,6","263,6"
3,4,A,Floresta Ombrófila Aberta,Floresta,2,1,Mata Atlântica,"597.989,6","290.130,6","181.066,5",...,"73.348,4","73.348,4","73.346,4","73.344,8","73.344,8","73.340,1","73.208,9","73.162,1","73.094,1","73.026,2"
4,5,Aa,Floresta Ombrófila Aberta,Floresta,3,1,Amazônia,"47.151,3","45.089,1","43.842,8",...,"42.397,0","42.397,0","42.389,2","42.347,2","42.340,0","42.322,7","42.312,9","42.298,1","42.284,5","42.270,9"
